In [1]:
## author: Xiaowu He. horacehxw@gmail.com
%matplotlib inline
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import rand
from scipy.stats import norm
import seaborn as sns
sns.set(color_codes=True)

In [2]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [3]:
class Node:
    '''
    This is the class for the basic nodes in the information network.
    Each node represents a single person in social network, 
        they are connected within a given "Group", which represents city.
    
    @Attribute:
        population: float between [0,100], propotion of population in the Node know the message
        energy: importance of the energy recevied, decreament every time interval
        
    @Function:
        prob: return the probability to transmit message
        update: energy decreament
        
    
    '''
    
    def __init__(self):
        self.energy = 0
        self.know = False
        
    def get_info(self, info):
        self.energy = info
        self.know = True
        
    def prob(self):
        '''
        The probability of the Node to transmit message to connected Node
        Should be a monotonical nondecreasing function of message energy.
        
        The cumulative distribution function of normal distribution is applied.
            
        @Return:
            probability: float in [0,1]
        '''
        if self.know:
            #return norm.cdf(self.energy, 75, 50)
            return np.abs(self.energy / 100.)
        return 0
    
    def update(self):
        '''
        Natrual update w/o other influence for attribute in a given time interval
        
        * population multiply by (1+ratio)
        '''
        self.energy *= 0.8
        
        
    def transmit(self, other):
        if not other.know:
            other.get_info(self.energy+1)
        
        

In [10]:
class Group:
    '''
    A network of Nodes where each Node is connceted via the power law probability graph.
    Each node's
    @Attribute:
        nodes: list of Node object
        G: a graph shows the relationship between each object
    
    
    @Function:
        
    
    '''
    
    def __init__(self, num, opinion=False):
        '''
        Use the index of Node in the list 
        
        input:
            num: number of Node in this Group object
        '''
        self.nodes = [Node() for _ in range(num)]
        self.G = nx.powerlaw_cluster_graph(num, 2, 1) # 0.5 controls the prefrence of clusters
      
    def start(self, info):
        '''
        Let one random person know the message
        '''
        index = np.random.randint(0, len(self.nodes))
        self.nodes[index].get_info(info)
    
    def know(self):
        '''
        return the number of people know this message at the time period
        '''
        return np.sum([node.know for node in self.nodes])
    
    def energy(self):
        '''
        return the total energy of the message in this Group
        '''
        return np.sum([np.abs(node.energy) for node in self.nodes])
    
    def update(self):
        '''
        call update for each node.
        then transmit to the neigherbors with certain probability.
        '''
        for node in self.nodes:
            node.update()
        for i in range(len(self.nodes)):
            here = self.nodes[i]
            for j in self.G.adj[i]:
                there = self.nodes[j]
                if rand() < here.prob():
                    here.transmit(there) 
                    
    def status(self):
        energy = np.array([node.energy for node in self.nodes])
        knows = np.array([int(node.know)] for node in self.nodes)
        import pdb; pdb.set_trace()
        print('total known population:{:8}, total energy: {:12.2f} '.format(np.sum(knows), np.sum(energy)))
        return knows, energy

In [11]:
NY = Group(100)

In [ ]:
NY.status()

> <ipython-input-10-46d4ba3ba221>(62)status()
-> print('total known population:{:8}, total energy: {:12.2f} '.format(np.sum(knows), np.sum(energy)))
(Pdb) np.sum(knows)
<generator object Group.status.<locals>.<genexpr> at 0x7f52069a7830>


In [ ]:
def simulate(NY, info, time = 100):
    '''
    Single city offline model simulation.
    
    @Input:
        num: population of given group
        info: relative importance(initial energy) of infomation
        
    @Return:
        pop_know: The number of people know this message as a time series.
        energy_total: total energy of this message in the city as a time series.
        NY.G: the internal connection graph of ths city. used for topology analysis.
    '''
    NY.start(info)
    status = []
    for _ in range(time):
        NY.update()
        status.append(NY.status())
    return pop_know, energy_total, NY.G, NY


status = []
    for _ in range(time):
        status.append(net.status())
        net.update()
    time, pop, energy = zip(*status)
    return list(time), np.vstack(pop), np.vstack(energy)


In [ ]:
pop, energy, G, NY = simulate(Group(1000), 20)

In [ ]:
iplot([go.Scatter(y=pop), go.Scatter(y=energy)])

In [ ]:
#G = nx.powerlaw_cluster_graph(100000, 3, 0.5)
sns.distplot(np.log(list(dict(nx.degree(G)).values())/np.log(3)),kde=False)
plt.title("distribution of degree of nodes (log scale), n=10^5")
plt.xlabel('log3(degree)')
plt.ylabel('counts of given scale')

In [ ]:
def generate_traces(G, layout_funct=nx.spring_layout):
    pos = layout_funct(G)
    N = len(G.nodes())
    
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=go.Line(width=0.1,color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in G.edges():
        edge_trace['x'] += [pos[edge[0]][0],pos[edge[1]][0], None]
        edge_trace['y'] += [pos[edge[0]][1],pos[edge[1]][1], None]

    node_trace = go.Scatter(
        x = [pos[k][0] for k in range(N)],
        y = [pos[k][1] for k in range(N)],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=go.Marker(
            showscale=True,
            # colorscale options
            # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
            # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
            colorscale='Jet',
            reversescale=True,
            color=[],
            size=5,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            #line=dict(width=1)
        ))
    
    # function to color the nodes    
    for i in range(N):
        node_trace['marker']['color'].append(len(G.adj[i]))
        node_info = 'degree of node: '+str(len(G.adj[i]))
        node_trace['text'].append(node_info)

    return [edge_trace, node_trace] # node is plot above the edges !!!

In [ ]:
layout=go.Layout(
                title='Energy Visualization',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=go.XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=go.YAxis(showgrid=False, zeroline=False, showticklabels=False))

In [ ]:
G = nx.powerlaw_cluster_graph(500, 3, 0.5)
data = generate_traces(G)#layout_funct=nx.fruchterman_reingold_layout)
py.iplot(data, layout=layout)

In [ ]:
from numpy.random import choice
class GroupCenter(Group):
    '''
    The Group with extra centralize message propagation method, etc. we take into 
        account the traditional authority medias, like TV, radio, or News app.
    
    There's a popularity term defines the popularity of centralized medias inside 
        the group. After a certain time interval, we imagine the centralized medias 
        transmit the infomation to those.
        
    The Group itself can't control the transmission of mass media, since they are mostly
        across multiple cities.
        
    @Constant:
        PROB_ACCEPT: 
            acceptance percentage of people who received message from the 
            centralized medias.
            
        
    @Attribute:
        attention: a boolean indicating wheter the local media notice the message or not.
            need to be something with huge energy in the city.
        index_central: index of Nodes who receives message from centers.(Fixed after Initialize)
        popularity: percentage of person who can receive the message from centers.
    '''
    PROB_ACCEPT = 0.1
    ENERGY_THRESHOLD_AVG = 2 #the total energy 5 times the population, which is huge.
    
    def __init__(self, num, popularity=0.3, name=""):
        '''
        This group has some index of people who receive messages from the centralized medias.
        
        The index is chosen by the popularity.
        '''
        super().__init__(num)
        self.popularity = popularity
        self.index_central = choice(num, int(num*popularity))
        self.attention = False
        self.name = name
        
    def start(self, info):
        self.info = info #record the initial infomation
        super().start(info)
        
    def transmit_center(self, info):
        '''
        Specify the behavior if the mass centralized media spread the infomation in the city.
        '''
        self.attention = False #switch to moniter the city's energy in the next stage
        for idx in self.index_central:
            if rand() < self.PROB_ACCEPT and not self.nodes[idx].know:
                self.nodes[idx].get_info(info)
        
    
    def update(self):
        super().update()
        if self.energy() >=  len(self.nodes) * self.ENERGY_THRESHOLD_AVG:
            self.attention = True
#         #TO DO: There should be some other threshold to 
#         #       determine whether to public this or not
#         if self.time % self.TIME_INTERVAL_CENTER == 0 and self.attention:
#             print('Centralized Media Public at time {}'.format(self.time))
            

In [ ]:
pop, energy, G = simulate(GroupCenter(100000, popularity=0.5), 30)

In [ ]:
iplot([go.Scatter(y=pop[0]), go.Scatter(y=energy[0])])

In [ ]:
class GroupNet():
    '''
    Represents the network containing many cities.
    '''
    
    TIME_MEDIA = 3
    TIME_CROSS_CITY = -1
    
    def __init__(self, groups,  center=True):
        self.time = 0
        self.center = center
        self.groups = []
        self.attention = False
        for group in groups:
            self.groups.append(group)
            
    def start(self, info, loc=None):
        '''
        Start at a random Node in a random city
        '''
        self.info = info
        if loc==None:
            idx = np.random.randint(0, len(self.groups))
        else:
            idx = loc
        print('simulation start at city {}'.format(self.groups[idx].name))
        self.groups[idx].start(info)
    
    def update(self):
        '''
        What to do in a time interval:
            call update for every city.
            check whether or not start the mass media transmission.
        '''
        self.time += 1
        for group in self.groups:
            group.update()
            self.attention |= group.attention
#         self.groups = Parallel(n_jobs=-1)(delayed(funct)(group) for group in self.groups)
            
        # spread across the cities
        if self.time == self.TIME_CROSS_CITY: #only one time for each message
            for group in self.groups:
                if group.know() == 0:
                    group.start(self.info)
        
        # spread by centralized medias
        if self.center and self.time % self.TIME_MEDIA == 0 and self.attention:
            self.attention =False
            for group in self.groups:
                group.transmit_center(self.info)
                
        
    def status(self):
        '''
        report the current status of network
        
        @Return:
            time: scalar, time stamp
            knows: 1D array for cities
            energy 1D array
        '''
        energy = np.array([group.energy() for group in self.groups])
        knows = np.array([group.know() for group in self.groups])
        print("time:{:3}    total known population:{:8}, total energy: {:12.2f}, attention={} ".format(self.time,
                                                                                                       np.sum(knows), 
                                                                                                       np.sum(energy),
                                                                                                       self.attention))
        return self.time, knows, energy

In [ ]:
def simulate_cities(net, info,loc=None, time = 100):
    '''
    Multiple city offline model with mass media simulation.
    
    @Input:
        net: a GroupNet object with multiple cities.
        info: relative importance(initial energy) of infomation
        
    @Return:
        time:
            1D array, time index
        know: 
            2D array, The number of people know this message as multiple time series
        energy_total:
            2D array, total energy of this message in the city as multiple time series.
    '''
    net.start(info,loc)
    status = []
    for _ in range(time):
        status.append(net.status())
        net.update()
    time, pop, energy = zip(*status)
    return list(time), np.vstack(pop), np.vstack(energy)


In [ ]:
# groups = [GroupCenter(1000, 0.9), GroupCenter(1000, 0.9), GroupCenter(1000, 0.9)]
# net = GroupNet(groups)
# time, know, energy = simulate_cities(net, 40)

## Deal with real world data: Population in US cities in 1870...

In [ ]:
import pandas as pd

In [ ]:
city_pop_1870 = pd.read_csv('1870_us_city_pop.csv', index_col=0)
city_geo = pd.read_json("https://gist.githubusercontent.com/Miserlou/c5cd8364bf9b2420bb29/raw/2bf258763cdddd704f8ffd3ea9a3e81d25e2c6f6/cities.json")
city_geo = city_geo[['city','state', 'latitude', 'longitude']]
merged = city_pop_1870.merge(city_geo, how='inner', left_on=['City', 'State'], right_on=['city', 'state'])
df_1870 = merged.drop(['City', 'State'], axis=1)
df_1870["1870 Population"] = pd.to_numeric(df_1870['1870 Population'].str.replace(",", ""))

In [ ]:
# Philadelphia: Lincoln's speech
groups = [GroupCenter(int(df_1870['1870 Population'][i]/10), popularity=0.36,name=df_1870['city'][i]) for i in range(40)]
net = GroupNet(groups)
time, know, energy = simulate_cities(net, 37, loc=1, time=30)

In [ ]:
fig = city_slider(df_1920.iloc[:40], energy, "Information Hot Degree Distribution", "Average Hot Degree")
py.iplot(fig)

In [ ]:
v_layout = {
    'shapes': [
        # Line Vertical
        {
            'type': 'line',
            'x0': 8,
            'y0': 0,
            'x1': 8,
            'y1': 1000000,
            'line': {
                'color': 'rgb(0, 255,0)',
                'width': 1,
            },
        },
        {
            'type': 'line',
            'x0': 12,
            'y0': 0,
            'x1': 12,
            'y1': 2000000,
            'line': {
                'color': 'rgb(255,0,0)',
                'width': 1,
            },
        },
        {
            'type': 'line',
            'x0': 24,
            'y0': 0,
            'x1': 24,
            'y1': 2000000,
            'line': {
                'color': 'rgb(255,0,0)',
                'width': 1,
            },
        },
        {
            'type': 'line',
            'x0': 36,
            'y0': 0,
            'x1': 36,
            'y1': 2000000,
            'line': {
                'color': 'rgb(255,0,0)',
                'width': 1,
            },
        }
    ]
}


In [ ]:
plt.plot(np.array(time)*2, np.sum(know, axis=1))
plt.title('known population trend')
plt.xlabel('time in hours (12h a day)')
plt.ylabel('Total population know the news')
plt.axvline(x=8, color='g')
plt.axvline(x=12, color='r')
plt.axvline(x=24, color='r')
plt.axvline(x=36, color='r')

In [ ]:
plt.plot(np.array(time)*2, np.sum(energy, axis=1))
plt.title('total enery trend')
plt.xlabel('time in hours (12h a day)')
plt.ylabel('Total energy over all cities')
plt.axvline(x=8, color='g')
plt.axvline(x=12, color='r')
plt.axvline(x=24, color='r')
plt.axvline(x=36, color='r')

In [ ]:
fig = {'data' :[go.Scatter(x=np.array(time)*2,y = energy[:,i], opacity=0.6, name=df_1870['city'][i]) for i in range(know.shape[1])],
       'layout' :v_layout}
iplot(fig)

In [ ]:
fig = {'data' :[go.Scatter(x=np.array(time)*2,y = know[:,i], opacity=0.6, name=df_1870['city'][i]) for i in range(know.shape[1])],
       'layout' :v_layout}
iplot(fig)

In [ ]:
np.sum(df_1870['1870 Population'])

In [ ]:
class GroupFake():
    '''
    Fake Group class to imitate the behavior of cities with close population.
    '''
    
    def __init__(self, ref):
        self.ref = ref # ref must be a real Group Object!
        self.attention = False
    
    def start(self, info):
        self.attention = ref.attention
    
    def update(self, info):
        pass
    
    def transmit_center(self, info):
        pass
    
    def know(self):
        return ref.know()
    
    def energy(self):
        return ref.energy()
    
    def status(self):
        pass

## Deal with real world data: Population in US cities in 1920...

In [ ]:
merged = city_pop_1870.merge(city_geo, how='inner', left_on=['City', 'State'], right_on=['city', 'state'])
df_1870 = merged.drop(['City', 'State'], axis=1)
df_1870["1870 Population"] = pd.to_numeric(df_1870['1870 Population'].str.replace(",", ""))

In [ ]:
city_pop_1920 = pd.read_csv('1920_us_city_pop.csv', index_col=0)
merged = city_pop_1920.merge(city_geo, how='inner', left_on=['City', 'State'], right_on=['city', 'state'])
df_1920 = merged.drop(['City', 'State'], axis=1)
df_1920['Population'] = pd.to_numeric(df_1920['Population'].str.replace(",", ""))

In [ ]:
df_1920.head()

In [ ]:
df_1870['Population'] = df_1870['1870 Population']

In [ ]:
# NY wall street terrorist attack
groups = [GroupCenter(int(df_1920['Population'][i]/10), popularity=0.46,name=df_1920['city'][i]) for i in range(40)]
net = GroupNet(groups)
time, know, energy = simulate_cities(net, 63.9, loc=0, time=30)

In [ ]:
fig = city_slider(df_1870.iloc[:40], know, "Population know", "average number know the news")
py.iplot(fig)

In [ ]:
fig = city_slider(df_1870.iloc[:40], know, "Population know", "average number know the news")
py.iplot(fig)

In [ ]:
plt.plot(np.array(time)*2, np.sum(know, axis=1))
plt.title('known population trend')
plt.xlabel('time in hours (12h a day)')
plt.ylabel('Total population know the news')
plt.axvline(x=8, color='g')
plt.axvline(x=12, color='r')
plt.axvline(x=24, color='r')
plt.axvline(x=36, color='r')
plt.axvline(x=48, color='r')

In [ ]:
plt.plot(np.array(time)*2, np.sum(energy, axis=1))
plt.title('total enery trend')
plt.xlabel('time in hours (12h a day)')
plt.ylabel('Total energy over all cities')
plt.axvline(x=8, color='g')
plt.axvline(x=12, color='r')
plt.axvline(x=24, color='r')
plt.axvline(x=36, color='r')
plt.axvline(x=48, color='r')

In [ ]:
fig = {'data' :[go.Scatter(x=np.array(time)*2,y = know[:,i], opacity=0.6, name=df_1920['city'][i]) for i in range(know.shape[1])],
       'layout' :v_layout}
py.iplot(fig)

In [ ]:
fig = {'data' :[go.Scatter(x=np.array(time)*2,y = energy[:,i], opacity=0.6, name=df_1920['city'][i]) for i in range(know.shape[1])],
       'layout' :v_layout}
py.iplot(fig)

In [ ]:
def city_fig(df, target, title="", color_title=""):
    text = df['city'] + '<br>Population ' + (df['Population']/1e6).astype(str)+' million'
    scale = 7000

    for i in range(len(limits)):
        city = dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df['longitude'],
            lat = df['latitude'],
            text = text,
            marker = dict(
                size = df['Population']/scale,
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area',
                showscale=True,
                # colorscale options
                # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
                # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
                colorscale='Jet',
                reversescale=False,
                color=target/df['Population'],
                colorbar=dict(
                    thickness=15,
                    title=color_title,
                    xanchor='left',
                    titleside='right'
                )
            ))

    layout = dict(
            title = title,
            showlegend = True,
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(217, 217, 217)',
                subunitwidth=1,
                countrywidth=1,
                subunitcolor="rgb(255, 255, 255)",
                countrycolor="rgb(255, 255, 255)"
            ),
        )

    fig = dict( data=[city], layout=layout )
    return fig, city, layout

In [ ]:
fig, _, _ = city_fig(df_1920, df_1920['Population'],\
                     "Population distribution", "Population")
py.iplot(fig)

In [ ]:
def city_slider(df, target, title, color_title):
    data = []
    for energy_snap in target:
        _, data_snap, layout = city_fig(df, energy_snap,title, color_title)
        data_snap['visible'] = False
        data.append(data_snap)
        
    steps = []
    for i in range(len(data)):
        step = dict(
            method = 'restyle',
            args = ['visible', [False] * len(data)],
        )
        step['args'][1][i] = True # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active = 10,
        currentvalue = {"prefix": "Time (2 Hour): "},
        pad = {"t": 50},
        steps = steps
    )]
    layout['sliders'] = sliders
    fig = dict(data=data, layout=layout)
    return fig

In [ ]:
fig = city_slider(df_1920.iloc[:40], energy, "Information Hot Degree Distribution", "Average Hot Degree")
py.iplot(fig)

In [ ]:
fig = city_slider(df_1920.iloc[:40], know, "Population know", "average number know the news")
py.iplot(fig)

## Internet Workd: One world, One city

In [ ]:
world = GroupNet([GroupCenter(3030000, popularity=0.4)])

In [ ]:
world = GroupNet([GroupCenter(3030000, popularity=0.4)])
time, know, energy = simulate_cities(world, 68.64, loc=0, time=30)

In [ ]:
plt.plot(np.array(time)*2, np.sum(energy, axis=1))
plt.title('total enery trend')
plt.xlabel('time in hours (12h a day)')
plt.ylabel('Total energy over all cities')

In [ ]:
plt.plot(np.array(time)*2, np.sum(know, axis=1) / 3030000)
plt.title('known population trend')
plt.xlabel('time in hours (12h a day)')
plt.ylabel('Proportion of population know the news')

In [ ]:
google_trend = pd.read_csv('multiTimeline.csv').iloc[20:100]

In [ ]:
google_trend.plot(x='Date', legend=False)
plt.ylabel('google search index')

In [ ]:
google_trend.shape

## 2050: ACCEPT=0.1 and Change of topology

In [ ]:
G = nx.powerlaw_cluster_graph(200, 2, 1)

In [ ]:
world = GroupNet([GroupCenter(330000, popularity=0.3)])

In [ ]:
time, know, energy = simulate_cities(world, 68.64, loc=0, time=60)

In [ ]:
plt.plot(np.array(time)*2,know)
plt.title('known population trend')
plt.xlabel('time in hours (12h a day)')
plt.ylabel('Total population know the news')

In [ ]:
plt.plot(np.array(time)*2, np.sum(energy, axis=1))
plt.title('total enery trend')
plt.xlabel('time in hours (12h a day)')
plt.ylabel('Total energy over all cities')

## Opinion Change!

In [ ]:
from numpy.random import rand
class NodeOpinion(Node):
    def __init__(self, prob):
        '''
        prob: probability that this node is a ciritcer
        '''
        self.criticize = rand()<prob
        self.energy = 0
        self.know = False
        
    def transmit(self, other):
        if not other.know:
            other.get_info(self.energy+1)
            
    def get_info(self, info):
        if not self.know:
            self.know = True
            if self.criticize:
                self.energy = -info
            else:
                self.energy = info
        elif np.abs(self.energy) < np.abs(info):
            self.energy = (self.energy + info) / 2.
     
    def transmit(self, other):
        other.get_info(self.energy + np.sign(self.energy))

In [ ]:
class GroupCenterOpinion(GroupCenter):
    def __init__(self,num, criticize=0.4, popularity=0.3):
        self.popularity = popularity
        self.index_central = choice(num, int(num*popularity))
        self.attention = False
        self.nodes = [NodeOpinion(criticize) for _ in range(num)]
        self.G = nx.powerlaw_cluster_graph(num, 2, 1) # 0.5 controls the prefrence of clusters
        self.name = "ha"

    def energy_status(self):
        pos = 0
        neg = 0
        for node in self.nodes:
            if node.energy>0:
                pos += node.energy
            else:
                neg += node.energy
        return pos, neg
    
    def know_status(self):
        pos = 0
        neg = 0
        for node in self.nodes:
            if node.energy>0:
                pos += 1
            elif node.energy < 0:
                neg += 1
        return pos, neg

In [ ]:
city = GroupCenterOpinion(1000)

In [ ]:
city.start(100)

In [ ]:
a = 0
for node in city.nodes:
    if node.know:
        a = node

In [ ]:
a.energy

In [ ]:
a.know

In [ ]:
class GroupNetOp(GroupNet):
    def status(self):
        energy = np.array([self.groups[0].energy_status()])
        knows = np.array([self.groups[0].know_status()])
        print("time:{:5} energy:{}, pop:{} ".format(self.time, self.groups[0].energy(), self.groups[0].know()))
        return self.time, knows, energy

In [ ]:
world = GroupNetOp([GroupCenterOpinion(30300, popularity=0.4)])

In [ ]:
time, know, energy = simulate_cities(world, 68.64, loc=0, time=60)

In [ ]:
legend = ['positive attitude', 'negative attitude']

In [ ]:
fig = {'data' :[go.Scatter(x=np.array(time)*2,y = np.abs(energy[:,i]), opacity=0.6, name=legend[i]) for i in range(know.shape[1])]}
py.iplot(fig)

In [ ]:
fig = {'data' :[go.Scatter(x=np.array(time)*2,y = know[:,i]*10000, opacity=0.6, name=legend[i]) for i in range(know.shape[1])]}
py.iplot(fig)